In [1]:
!pip install bertopic

In [2]:
from bertopic import BERTopic

c:\Users\ve797\anaconda4\lib\site-packages\umap\distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
c:\Users\ve797\anaconda4\lib\site-packages\umap\distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
c:\Users\ve797\anaconda4\lib\site-packages\umap\distances.py:1086: NumbaDeprecationWarning: The 'nopython' keywo

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
import re

import string

import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('stopwords') 

nltk.download('wordnet') 
nltk.download('omw-1.4')

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC
from sklearn.decomposition import TruncatedSVD
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score,train_test_split
from sklearn.metrics import accuracy_score,classification_report
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from imblearn.under_sampling import RandomUnderSampler



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ve797\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ve797\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ve797\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\ve797\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [24]:
import imblearn
print(imblearn.__version__)

0.10.1


## Load Dataset

In [2]:
data = pd.read_csv(
    "sentiment_tweets3.csv")
print(data.shape)
data.head()


(10314, 3)


,Index,message to examine,label (depression result)
0,106,just had a real good moment. i missssssssss hi...,0
1,217,is reading manga http://plurk.com/p/mzp1e,0
2,220,@comeagainjen http://twitpic.com/2y2lx - http:...,0
3,288,@lapcat Need to send 'em to my accountant tomo...,0
4,540,ADD ME ON MYSPACE!!! myspace.com/LookThunder,0


In [3]:
data['label (depression result)'].value_counts()

0    8000
1    2314
Name: label (depression result), dtype: int64

In [4]:
data.isna().sum()

Index                        0
message to examine           0
label (depression result)    0
dtype: int64

In [5]:
#check the shape of your dataset
print(data.shape)

(10314, 3)


## Data Preprocessing

- **Convert to lowercase**
- **Remove special characters**
- **Remove stopwords**
- **Lemmatization**
- 

### drop the index column

In [6]:
data = data.drop('Index', axis=1)
data.head()

,message to examine,label (depression result)
0,just had a real good moment. i missssssssss hi...,0
1,is reading manga http://plurk.com/p/mzp1e,0
2,@comeagainjen http://twitpic.com/2y2lx - http:...,0
3,@lapcat Need to send 'em to my accountant tomo...,0
4,ADD ME ON MYSPACE!!! myspace.com/LookThunder,0


### Remove special characters

In [7]:
stop_words = stopwords.words('english') 
print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [8]:
punctuation_list = list(string.punctuation)
print(punctuation_list)

['!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~']


In [9]:
useless_words = stop_words + punctuation_list

In [10]:
# remove punctuation and stopwords in review_text column

data["message to examine"] = data["message to examine"].str.lower().str.replace("[^\w\s]", "", regex=True).apply(
    lambda x: " ".join(word for word in word_tokenize(x) if word not in useless_words)
)

data["message to examine"].head(10)


0                   real good moment missssssssss much
1                     reading manga httpplurkcompmzp1e
2    comeagainjen httptwitpiccom2y2lx httpwwwyoutub...
3    lapcat need send em accountant tomorrow oddly ...
4                    add myspace myspacecomlookthunder
5                     sleepy good times tonight though
6    silkcharm nbn someone already said fiber home ...
7                         23 24ï½c possible today nice
8                       nite twitterville workout ciao
9                   dananner night darlin sweet dreams
Name: message to examine, dtype: object

### Lemmatization

In [11]:
# apply lemmatization to message to examine column

lemmatizer = WordNetLemmatizer()

data["message to examine"] = data["message to examine"].apply(
    lambda x: " ".join(lemmatizer.lemmatize(word) for word in word_tokenize(x))
)

data.head()

,message to examine,label (depression result)
0,real good moment miss much,0
1,reading manga httpplurkcompmzp1e,0
2,comeagainjen httptwitpiccom2y2lx httpwwwyoutub...,0
3,lapcat need send em accountant tomorrow oddly ...,0
4,add myspace myspacecomlookthunder,0


In [12]:
data["message to examine"].apply(
    lambda x: " ".join(tag for _, tag in nltk.pos_tag(word_tokenize(x)))
)

0                                        JJ JJ NN NNS RB
1                                              VBG NN NN
2                                               NN NN NN
3        NNS VBP VBP JJ NN NN RB VBD RB VBG NN VBG NN IN
4                                               VB NN NN
                              ...                       
10309                  NN NN NN NN NN VBN VBG NNS VBP VB
10310                       NN NNS NN VBP NN IN NN RB RB
10311                         NN NN NN NN NN NN IN NN NN
10312                               JJ NN JJ NN NN NN NN
10313                                                 CD
Name: message to examine, Length: 10314, dtype: object

### Topic Modelling

In [14]:
# Instantiate the BERTopic model and fit it to the data:
topic_model = BERTopic(language="english", calculate_probabilities=True)
topics, probs = topic_model.fit_transform(data["message to examine"])

In [15]:
freq = topic_model.get_topic_info(); freq.head(5)

,Topic,Count,Name
0,-1,4900,-1_depression_im_like_day
1,0,475,0_dinner_coffee_drink_food
2,1,372,1_twitter_tweet_twittering_tweeting
3,2,308,2_thanks_thank_welcome_haha
4,3,262,3_song_listening_album_music


In [16]:
# top words associated with the last topic in the model,
topic_model.get_topic(-1)

[('depression', 0.007791189705343569),
 ('im', 0.006455448354397425),
 ('like', 0.005705797649531151),
 ('day', 0.0055971807725498654),
 ('love', 0.005497859208390792),
 ('good', 0.005489308767404855),
 ('get', 0.005359212752218865),
 ('time', 0.005330010245570211),
 ('lol', 0.005165177092589418),
 ('go', 0.00514631936747867)]

In [17]:
topic_model.get_topic(0)  # Select the most frequent topic

[('dinner', 0.023168859093430343),
 ('coffee', 0.021725922154722598),
 ('drink', 0.020039571863826558),
 ('food', 0.019613283207736393),
 ('drinking', 0.01746108949415276),
 ('eat', 0.01688047972261368),
 ('tea', 0.01649102896669983),
 ('beer', 0.015959169687960358),
 ('hangover', 0.015602122420780054),
 ('breakfast', 0.015337951593272461)]

In [18]:
# Visualize the topics using the built-in visualization tool:
topic_model.visualize_topics()

In [19]:
topic_model.visualize_distribution(probs[200], min_probability=0.015)

### Feature Engineering

In [13]:
# Splitting into x and y values
X = data.drop('label (depression result)', axis=1)
y = data['label (depression result)']

In [14]:
X.head()

,message to examine
0,real good moment miss much
1,reading manga httpplurkcompmzp1e
2,comeagainjen httptwitpiccom2y2lx httpwwwyoutub...
3,lapcat need send em accountant tomorrow oddly ...
4,add myspace myspacecomlookthunder


In [15]:
X.shape

(10314, 1)

In [17]:
y.shape

(10314,)

In [18]:
# vectorize the training dataset
vectorizer = TfidfVectorizer(stop_words='english',use_idf=True)
X_idf = vectorizer.fit_transform(X['message to examine'])

In [19]:
# split the dataset into training and test dataset
X_train,X_test,y_train,y_test = train_test_split(X_idf,y,test_size=0.3, random_state=42)

In [20]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(7219, 22184) (7219,)
(3095, 22184) (3095,)


In [21]:
smote = SMOTE(random_state=20)
under = RandomUnderSampler(random_state=1)

steps = [('o', smote), ('u', under)]
pipeline = Pipeline(steps=steps)

X_train_res, y_train_res = pipeline.fit_resample(X_train, y_train)

AttributeError: 'NoneType' object has no attribute 'split'

In [27]:
# # Let's apply transformations to testing data
# X_test_tfidf = vectorizer.transform(X_test)


### Build classifier model 

In [28]:

svc_tfidf = Pipeline([('tf_idf_vectorizer',TfidfVectorizer(stop_words='english')),
                      ('svc',SVC(kernel='linear'))])

scores = cross_val_score(svc_tfidf,X_train,y_train,cv=2).mean()
print('Score',scores)

ValueError: 
All the 2 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\ve797\anaconda4\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\ve797\anaconda4\lib\site-packages\imblearn\pipeline.py", line 297, in fit
    self._final_estimator.fit(Xt, yt, **fit_params_last_step)
  File "c:\Users\ve797\anaconda4\lib\site-packages\sklearn\svm\_base.py", line 192, in fit
    X, y = self._validate_data(
  File "c:\Users\ve797\anaconda4\lib\site-packages\sklearn\base.py", line 584, in _validate_data
    X, y = check_X_y(X, y, **check_params)
  File "c:\Users\ve797\anaconda4\lib\site-packages\sklearn\util

In [ ]:
# KnnClassifier
knn_classify = KNeighborsClassifier()
knn_classify.fit(X_train_tfidf,y_train)



ValueError: Found input variables with inconsistent numbers of samples: [1, 7219]

In [ ]:
# Get test set predictions and evaluate model metrics
preds = knn_classify.predict(X_test_tfidf)
score = accuracy_score(y_test,preds)
classify_report = classification_report(y_test,preds)
print(score)
print(classify_report)
